In [13]:
# Import necessary functions
import os
import aryn_sdk
from aryn_sdk.partition import partition_file, draw_with_boxes, convert_image_element
import json
import pdf2image
#from openai import OpenAI
#from pydantic import BaseModel
#from openai.lib._parsing import type_to_response_format_param as pydantic_to_response_format
#from google.colab import userdata
from dotenv import load_dotenv
load_dotenv()

In [20]:
# Set your secrets in the colab notebook. Navigate to the left pane
# and choose the key option to set your keys. Make sure to enable Notebook access

# Visit https://www.aryn.ai/get-started to get a key.
aryn_api_key = os.getenv('aryn_API_KEY')
# Visit https://platform.openai.com/api-keys to get a key.
openai_api_key = os.getenv('anthropic_API_KEY')

print(aryn_api_key, openai_api_key)

None None


In [21]:
file_name = 'syllabus206.pdf'
file = open(file_name, 'rb')

#show the pdf
# pdf2image.convert_from_path(file_name)[0]

In [16]:
## Make a call to the partitioning service and set extract_images to true.
partitioned_file = partition_file(file, aryn_api_key, extract_images=True, extract_table_structure=True, use_ocr=True)

# show the pdf with bounding boxes
draw_with_boxes(file_name, partitioned_file)[0]

HTTPError: Error: status_code: 400, reason: {"detail":"Invalid API Key."}